In [24]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

24/02/19 11:47:49 INFO BlockManagerInfo: Removed broadcast_18_piece0 on ubuntu:41503 in memory (size: 20.6 KiB, free: 2.2 GiB)


import $ivy.$
import org.apache.log4j.{Level, Logger}

In [25]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [26]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDF")
                .getOrCreate()

import spark.implicits._

spark: SparkSession = org.apache.spark.sql.SparkSession@5d25b746
import spark.implicits._

In [27]:
val data = Seq((1, "Michael", 3000), (2, "Andy", 4500), (3, "Justin", 3500), (4, "Berta", 4000)).toDF("id", "name", "salary")
data.show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: DataFrame = [id: int, name: string ... 1 more field]

## UDF без аргументов

In [28]:
val random = udf(() => java.lang.Math.random())
data.withColumn("random", random()).show()

+---+-------+------+--------------------+
| id|   name|salary|              random|
+---+-------+------+--------------------+
|  1|Michael|  3000|0.013275872379206133|
|  2|   Andy|  4500|  0.6324165603391154|
|  3| Justin|  3500|   0.662683480202376|
|  4|  Berta|  4000|  0.3145871145867617|
+---+-------+------+--------------------+



random: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  f = ammonite.$sess.cell28$Helper$$Lambda$7434/0x0000000841e88040@24c6a270,
  dataType = DoubleType,
  inputEncoders = List(),
  outputEncoder = Some(
    value = ExpressionEncoder(
      objSerializer = BoundReference(
        ordinal = 0,
        dataType = DoubleType,
        nullable = false
      ),
      objDeserializer = AssertNotNull(
        child = UpCast(
          child = GetColumnByOrdinal(ordinal = 0, dataType = DoubleType),
          target = DoubleType,
          walkedTypePath = List("- root class: \"double\"")
        ),
        walkedTypePath = List("- root class: \"double\"")
      ),
      clsTag = Double
    )
  ),
  name = None,
  nullable = false,
  deterministic = true
)

## UDF с одним аргументом

In [29]:
val plusOne = udf((x: Int) => x + 1)
data.select($"name", $"salary", plusOne($"salary").as("new_salary")).show()

+-------+------+----------+
|   name|salary|new_salary|
+-------+------+----------+
|Michael|  3000|      3001|
|   Andy|  4500|      4501|
| Justin|  3500|      3501|
|  Berta|  4000|      4001|
+-------+------+----------+



plusOne: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  f = ammonite.$sess.cell29$Helper$$Lambda$7437/0x0000000841e86840@6e860f01,
  dataType = IntegerType,
  inputEncoders = List(
    Some(
      value = ExpressionEncoder(
        objSerializer = BoundReference(
          ordinal = 0,
          dataType = IntegerType,
          nullable = false
        ),
        objDeserializer = AssertNotNull(
          child = UpCast(
            child = GetColumnByOrdinal(ordinal = 0, dataType = IntegerType),
            target = IntegerType,
            walkedTypePath = List("- root class: \"int\"")
          ),
          walkedTypePath = List("- root class: \"int\"")
        ),
        clsTag = Int
      )
    )
  ),
  outputEncoder = Some(
    value = ExpressionEncoder(
      objSerializer = BoundReference(
        ordinal = 0,
        dataType = IntegerType,
        nullable = false
      ),
      objDeserializer = AssertNotNull(
        child = UpCast(
          child = GetColu

## UDF с двумя аргументами

In [30]:
val strLenPlus = udf((x: String, y: Int) => x.length + y)

strLenPlus: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  f = ammonite.$sess.cell30$Helper$$Lambda$7440/0x0000000841e84040@42ba2cbf,
  dataType = IntegerType,
  inputEncoders = List(
    Some(
      value = ExpressionEncoder(
        objSerializer = StaticInvoke(
          staticObject = class org.apache.spark.unsafe.types.UTF8String,
          dataType = StringType,
          functionName = "fromString",
          arguments = List(
            BoundReference(
              ordinal = 0,
              dataType = ObjectType(cls = class java.lang.String),
              nullable = true
            )
          ),
          inputTypes = List(),
          propagateNull = true,
          returnNullable = false,
          isDeterministic = true
        ),
        objDeserializer = Invoke(
          targetObject = UpCast(
            child = GetColumnByOrdinal(ordinal = 0, dataType = StringType),
            target = StringType,
            walkedTypePath = List("- root class: \"

In [31]:
data.select($"name", length($"name"), strLenPlus($"name",  $"id").as("nameLenPlusId")).show()

+-------+------------+-------------+
|   name|length(name)|nameLenPlusId|
+-------+------------+-------------+
|Michael|           7|            8|
|   Andy|           4|            6|
| Justin|           6|            9|
|  Berta|           5|            9|
+-------+------------+-------------+



In [32]:
data.select($"name", length($"name"), strLenPlus($"name", lit(1)).as("nameLenPlus1")).show()

+-------+------------+------------+
|   name|length(name)|nameLenPlus1|
+-------+------------+------------+
|Michael|           7|           8|
|   Andy|           4|           5|
| Justin|           6|           7|
|  Berta|           5|           6|
+-------+------------+------------+



## UDF для выражений WHERE

In [33]:
val oneArgFilter= udf((n: Int) => { n > 2 })
data.where(oneArgFilter($"id")).show()

24/02/19 11:47:52 INFO CodeGenerator: Code generated in 5.435809 ms


+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



oneArgFilter: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  f = ammonite.$sess.cell33$Helper$$Lambda$7444/0x0000000841e81840@336f43ff,
  dataType = BooleanType,
  inputEncoders = List(
    Some(
      value = ExpressionEncoder(
        objSerializer = BoundReference(
          ordinal = 0,
          dataType = IntegerType,
          nullable = false
        ),
        objDeserializer = AssertNotNull(
          child = UpCast(
            child = GetColumnByOrdinal(ordinal = 0, dataType = IntegerType),
            target = IntegerType,
            walkedTypePath = List("- root class: \"int\"")
          ),
          walkedTypePath = List("- root class: \"int\"")
        ),
        clsTag = Int
      )
    )
  ),
  outputEncoder = Some(
    value = ExpressionEncoder(
      objSerializer = BoundReference(
        ordinal = 0,
        dataType = BooleanType,
        nullable = false
      ),
      objDeserializer = AssertNotNull(
        child = UpCast(
          child = Ge

## Производительность

In [34]:
val train = spark.read.option("inferSchema", true).option("header", true).csv("../data/train.csv").cache()

24/02/19 11:47:52 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/02/19 11:47:52 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/02/19 11:47:52 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 11:47:52 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#2337, None)) > 0)
24/02/19 11:47:52 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 198.7 KiB, free 2.2 GiB)
24/02/19 11:47:52 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 34.2 KiB, free 2.2 GiB)
24/02/19 11:47:52 INFO BlockManagerInfo: Added broadcast_19_piece0 in memory on ubuntu:41503 (size: 34.2 KiB, free: 2.2 GiB)
24/02/19 11:47:52 INFO SparkContext: Created broadcast 19 from csv at cell34.sc:1
24/02/19 11:47:52 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/02/19 11:47:52 INFO SparkContext: Starting job: csv a

train: Dataset[Row] = [row_id: int, timestamp: bigint ... 8 more fields]

In [35]:
train.show(5)

24/02/19 11:48:39 INFO SparkContext: Starting job: show at cell35.sc:1
24/02/19 11:48:39 INFO DAGScheduler: Got job 18 (show at cell35.sc:1) with 1 output partitions
24/02/19 11:48:39 INFO DAGScheduler: Final stage: ResultStage 24 (show at cell35.sc:1)
24/02/19 11:48:39 INFO DAGScheduler: Parents of final stage: List()
24/02/19 11:48:39 INFO DAGScheduler: Missing parents: List()
24/02/19 11:48:39 INFO DAGScheduler: Submitting ResultStage 24 (MapPartitionsRDD[81] at show at cell35.sc:1), which has no missing parents
24/02/19 11:48:39 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 26.4 KiB, free 2.2 GiB)
24/02/19 11:48:39 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory (estimated size 10.9 KiB, free 2.2 GiB)
24/02/19 11:48:39 INFO BlockManagerInfo: Added broadcast_23_piece0 in memory on ubuntu:41503 (size: 10.9 KiB, free: 2.2 GiB)
24/02/19 11:48:39 INFO SparkContext: Created broadcast 23 from broadcast at DAGScheduler.scala:1580
24/0

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       NULL|                          NULL|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [36]:
train.count()

24/02/19 11:48:39 INFO BlockManagerInfo: Removed broadcast_23_piece0 on ubuntu:41503 in memory (size: 10.9 KiB, free: 2.2 GiB)
24/02/19 11:48:39 INFO DAGScheduler: Registering RDD 86 (count at cell36.sc:1) as input to shuffle 6
24/02/19 11:48:39 INFO DAGScheduler: Got map stage job 19 (count at cell36.sc:1) with 44 output partitions
24/02/19 11:48:39 INFO DAGScheduler: Final stage: ShuffleMapStage 25 (count at cell36.sc:1)
24/02/19 11:48:39 INFO DAGScheduler: Parents of final stage: List()
24/02/19 11:48:39 INFO DAGScheduler: Missing parents: List()
24/02/19 11:48:39 INFO DAGScheduler: Submitting ShuffleMapStage 25 (MapPartitionsRDD[86] at count at cell36.sc:1), which has no missing parents
24/02/19 11:48:39 INFO MemoryStore: Block broadcast_24 stored as values in memory (estimated size 25.5 KiB, free 2.2 GiB)
24/02/19 11:48:39 INFO MemoryStore: Block broadcast_24_piece0 stored as bytes in memory (estimated size 11.4 KiB, free 2.2 GiB)
24/02/19 11:48:39 INFO BlockManagerInfo: Added bro

res36: Long = 101230332L

In [37]:
val to_month = udf((ts_ms: Long) => ts_ms / 31536000000L / 12) // 1 year is 31536000000 ms

to_month: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  f = ammonite.$sess.cell37$Helper$$Lambda$7451/0x0000000841e7a840@c073a89,
  dataType = LongType,
  inputEncoders = List(
    Some(
      value = ExpressionEncoder(
        objSerializer = BoundReference(
          ordinal = 0,
          dataType = LongType,
          nullable = false
        ),
        objDeserializer = AssertNotNull(
          child = UpCast(
            child = GetColumnByOrdinal(ordinal = 0, dataType = LongType),
            target = LongType,
            walkedTypePath = List("- root class: \"long\"")
          ),
          walkedTypePath = List("- root class: \"long\"")
        ),
        clsTag = Long
      )
    )
  ),
  outputEncoder = Some(
    value = ExpressionEncoder(
      objSerializer = BoundReference(
        ordinal = 0,
        dataType = LongType,
        nullable = false
      ),
      objDeserializer = AssertNotNull(
        child = UpCast(
          child = GetColumnByOrdinal(

In [38]:
val t1 = System.nanoTime

train
    .select($"content_id", to_month($"timestamp").as("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration1 = (System.nanoTime - t1) / 1e9d

24/02/19 11:48:40 INFO CodeGenerator: Code generated in 17.619773 ms
24/02/19 11:48:40 INFO DAGScheduler: Registering RDD 94 (show at cell38.sc:6) as input to shuffle 7
24/02/19 11:48:40 INFO DAGScheduler: Got map stage job 21 (show at cell38.sc:6) with 44 output partitions
24/02/19 11:48:40 INFO DAGScheduler: Final stage: ShuffleMapStage 28 (show at cell38.sc:6)
24/02/19 11:48:40 INFO DAGScheduler: Parents of final stage: List()
24/02/19 11:48:40 INFO DAGScheduler: Missing parents: List()
24/02/19 11:48:40 INFO DAGScheduler: Submitting ShuffleMapStage 28 (MapPartitionsRDD[94] at show at cell38.sc:6), which has no missing parents
24/02/19 11:48:40 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 49.8 KiB, free 2.2 GiB)
24/02/19 11:48:40 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 21.8 KiB, free 2.2 GiB)
24/02/19 11:48:40 INFO BlockManagerInfo: Added broadcast_26_piece0 in memory on ubuntu:41503 (size: 21.8 KiB, fr

+----------+----------+
|content_id|avg(month)|
+----------+----------+
|      4519|       0.0|
|      4818|       0.0|
|      5518|       0.0|
|     13285|       0.0|
|     12027|       0.0|
|       833|       0.0|
|      9427|       0.0|
|       496|       0.0|
|      5156|       0.0|
|      6336|       0.0|
|      2866|       0.0|
|      1959|       0.0|
|      7982|       0.0|
|     23336|       0.0|
|       148|       0.0|
|      1342|       0.0|
|      1088|       0.0|
|      1580|       0.0|
|       471|       0.0|
|      2122|       0.0|
+----------+----------+
only showing top 20 rows



t1: Long = 1187661095110960L
duration1: Double = 2.496076622

In [39]:
val t2 = System.nanoTime

train
    .select($"content_id", ($"timestamp" / 31536000000L / 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration2 = (System.nanoTime - t2) / 1e9d

24/02/19 11:48:43 INFO DAGScheduler: Registering RDD 102 (show at cell39.sc:6) as input to shuffle 8
24/02/19 11:48:43 INFO DAGScheduler: Got map stage job 23 (show at cell39.sc:6) with 44 output partitions
24/02/19 11:48:43 INFO DAGScheduler: Final stage: ShuffleMapStage 31 (show at cell39.sc:6)
24/02/19 11:48:43 INFO DAGScheduler: Parents of final stage: List()
24/02/19 11:48:43 INFO DAGScheduler: Missing parents: List()
24/02/19 11:48:43 INFO DAGScheduler: Submitting ShuffleMapStage 31 (MapPartitionsRDD[102] at show at cell39.sc:6), which has no missing parents
24/02/19 11:48:43 INFO MemoryStore: Block broadcast_28 stored as values in memory (estimated size 44.4 KiB, free 2.2 GiB)
24/02/19 11:48:43 INFO MemoryStore: Block broadcast_28_piece0 stored as bytes in memory (estimated size 19.5 KiB, free 2.2 GiB)
24/02/19 11:48:43 INFO BlockManagerInfo: Added broadcast_28_piece0 in memory on ubuntu:41503 (size: 19.5 KiB, free: 2.2 GiB)
24/02/19 11:48:43 INFO SparkContext: Created broadcast

+----------+--------------------+
|content_id|          avg(month)|
+----------+--------------------+
|      4519|0.015429677931821352|
|      4818|0.016230649272587192|
|      5518| 0.02103952664360606|
|     13285| 0.04141825984987046|
|     12027| 0.04150963056316861|
|       833|0.022584471034752644|
|      9427| 0.02158891106034129|
|       496|0.020155351740181845|
|      5156|0.018064186032779997|
|      6336|0.019195051162991618|
|      2866|0.026921431007821156|
|      1959| 0.02670915106444127|
|      7982| 0.02122819619071833|
|     23336|0.019260306662998706|
|       148|0.017910791660268703|
|      1342| 0.02195320830526077|
|      1088| 0.02461949633696784|
|      1580|0.022461073541947943|
|       471| 0.02611517946736722|
|      2122|0.026057020501248997|
+----------+--------------------+
only showing top 20 rows



t2: Long = 1187663824245904L
duration2: Double = 1.950988736

In [40]:
println(duration1, duration2)

(2.496076622,1.950988736)
